In [ ]:
import os
spark_home = os.path.abspath(os.getcwd() + "/spark/spark-3.5.5-bin-hadoop3")
hadoop_home = os.path.abspath(os.getcwd() + "/spark/winutils")
print(f"I am using the following SPARK_HOME: {spark_home}")
if os.name == 'nt':
    os.environ["HADOOP_HOME"] = f"{hadoop_home}"
    print(f"Windows detected: set HADOOP_HOME to: {os.environ['HADOOP_HOME']}")
    hadoop_bin = os.path.join(hadoop_home, "bin")
    os.environ["PATH"] = f"{hadoop_bin};{os.environ['PATH']}"
    print(f"  Also added Hadoop bin directory to PATH: {hadoop_bin}")

import findspark
import pyspark
from pyspark.streaming import StreamingContext

findspark.init(spark_home)
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession.builder.getOrCreate()


I am using the following SPARK_HOME: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\spark-3.5.5-bin-hadoop3
Windows detected: set HADOOP_HOME to: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\winutils
  Also added Hadoop bin directory to PATH: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\winutils\bin


In [ ]:
papers_df = spark.read.json("data/interim")


# Data structure

Understand what you have and whether it's clean:
- What is the schema of the dataset?
- Are there missing values in key fields like title, summary, main_category, categories, or published?
- Are there duplicated records (aid)?

In [ ]:
# Schema
papers_df.printSchema()

# Get basic statistics and information
print(f"Dataset size: {papers_df.count()} papers")
print(f"Number of columns: {len(papers_df.columns)}")

# Check for missing values
from pyspark.sql.functions import col, count, when, isnan, isnull
from pyspark.sql.functions import split, size, year, month, explode, round, sum as spark_sum

papers_df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) 
                          for c in papers_df.columns]).show()


In [ ]:
# Print the first record in a dictionary format with better formatting
from pprint import pprint

# Get the first record as a dictionary
first_record = papers_df.limit(1).toPandas().T.to_dict()[0]

# Pretty print with formatting
print("First record:")
pprint(first_record, width=100, compact=False)


# Main category distribution

Understand your labels before deciding on prediction strategy:
- How many unique values are there in `main_category`?
- What are the top N most frequent `main_category` values?
- Are some `main_category` values very rare or dominant? (e.g., does one class dominate 50% of the data?)
- How many unique `categories` values?
- What’s the average number of `categories` per paper (for multilabel setup)?
- How do `main_category` distributions differ over week?
- What's the distribution of Arxiv main category (before `.`)?


In [ ]:
# Target Analysis: Main Category Distribution and Characteristics# Target Analysis: Main Category Distribution and Characteristics

# ---------- 1. Count unique values in main_category ----------
unique_main_categories = papers_df.select("main_category").distinct().count()
print(f"Number of unique main_categories: {unique_main_categories}")


In [ ]:
# ---------- 2. Top N most frequent main_categories ----------
N = 10  # Change this to see more or fewer categories
main_cat_dist = papers_df.groupBy("main_category").count().orderBy("count", ascending=False)
print(f"\nTop {N} most frequent main categories:")
main_cat_dist.show(N, truncate=False)


In [ ]:
# ---------- 3. Check for category imbalance ----------
total_papers = papers_df.count()
main_cat_dist_with_pct = main_cat_dist.withColumn(
    "percentage", round((col("count") / total_papers) * 100, 2)
)

print("\nCategory distribution with percentages:")
main_cat_dist_with_pct.show(20, truncate=False)

# Find dominant categories (>10%) and rare categories (<1%)
dominant_categories = main_cat_dist_with_pct.filter("percentage > 5")
rare_categories = main_cat_dist_with_pct.filter("percentage < 1")

print(f"Dominant categories (>5%): {dominant_categories.count()}")
dominant_categories.show(truncate=False)

print(f"Rare categories (<1%): {rare_categories.count()}")
rare_categories.show(truncate=False)


In [ ]:
# ---------- 4. Unique subcategories in categories field ----------
# Assuming categories field contains comma-separated category codes
categories_exploded = papers_df.select(
    explode(split(col("categories"), ",")).alias("subcategory")
)
unique_subcategories = categories_exploded.select("subcategory").distinct().count()
print(f"\nNumber of unique subcategories: {unique_subcategories}")


In [ ]:
from pyspark.sql.functions import concat, lit

cat_dist_with_count = papers_df.withColumn(
    "num_categories", size(split(col("categories"), ","))
)

avg_labels = cat_dist_with_count.agg(
    {"num_categories": "avg"}
).collect()[0][0]

print(f"\nAverage number of categories per paper: {avg_labels:.2f}")

category_dist = cat_dist_with_count.groupBy("num_categories").count()

# Add percentage column
category_dist_with_pct = category_dist.withColumn(
    "percentage", round((col("count") / total_papers) * 100, 2)
)

# Format the output to include both count and percentage
print("\nDistribution of number of categories per paper:")
category_dist_with_pct.select(
    "num_categories", 
    "count", 
    concat(col("percentage").cast("string"), lit("%")).alias("percentage")
).orderBy("num_categories").show(20)


In [ ]:
# ---------- 6. Category distributions over week ----------
from pyspark.sql.functions import weekofyear, date_format, dayofweek, date_sub, expr, to_date

# Convert published field to date components and find the Monday that starts each week
papers_with_date = papers_df.withColumn(
    "pub_date", to_date(col("published"))
).withColumn(
    # Calculate days to subtract to get to Monday (Sunday=1, Monday=2, etc.)
    # For Monday (2) we subtract 0, Tuesday (3) subtract 1, etc.
    "days_from_monday", expr("dayofweek(pub_date) - 2")
).withColumn(
    # Handle Sunday (need to subtract 6 days instead of -1)
    "days_from_monday", 
    when(col("days_from_monday") < 0, 6).otherwise(col("days_from_monday"))
).withColumn(
    # Get the Monday date
    "week_start", date_sub(col("pub_date"), col("days_from_monday"))
).withColumn(
    # Format it as "Mar-31" 
    "week_date", date_format(col("week_start"), "MMM-d")
)

# Count papers by week and category
weekly_cat_dist = papers_with_date.groupBy("week_date", "main_category").count().orderBy("week_date", "count", ascending=False)

# print("\nCategory distribution by week (showing first 20 rows):")
# weekly_cat_dist.show(20, truncate=False)

# Calculate percentage distribution by week
weekly_totals = papers_with_date.groupBy("week_date").count().withColumnRenamed("count", "week_total")

weekly_cat_pct = weekly_cat_dist.join(weekly_totals, "week_date") \
    .withColumn("percentage", round((col("count") / col("week_total")) * 100, 2)) \
    .orderBy("week_date", "percentage", ascending=False)

# print("\nPercentage distribution of categories by week (showing first 20 rows):")
# weekly_cat_pct.select("week_date", "main_category", "count", "week_total", "percentage").show(20, truncate=False)

# Optional: Track specific categories over time
top_categories = main_cat_dist.limit(10).select("main_category").rdd.flatMap(lambda x: x).collect()
top_cat_trend = weekly_cat_pct.filter(col("main_category").isin(top_categories))

print("\nTrend for top 10 categories over time (by week):")
top_cat_trend.orderBy("week_date", "main_category").show(50, truncate=False)

# Add a visualization hint
print("\nNote: The week_date format is 'MMM-d' showing the Monday that starts each week")


In [ ]:
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas and prepare data
top_cat_pandas = top_cat_trend.orderBy("week_date", "main_category").toPandas()

# Create line chart with Plotly
fig = px.line(
    top_cat_pandas, 
    x='week_date', 
    y='percentage', 
    color='main_category',
    markers=True,
    title='Weekly Trends of Top 10 Categories',
    labels={'week_date': 'Week', 'percentage': 'Percentage (%)'}
)

# Basic layout improvements
fig.update_layout(
    xaxis=dict(tickangle=45),
    yaxis=dict(ticksuffix='%'),
    legend=dict(
        orientation='h',  # Horizontal orientation
        yanchor='top', # Anchor to bottom of legend box
        y=1.02,          # Position just above the plot area
        xanchor='center', # Center horizontally
        x=0.5
    ),
    margin=dict(t=50),   # Add top margin for legend space
    template='plotly_white'
)

# Show the plot
fig.show(render="png")


In [ ]:
# ---------- 7. Archive distribution ----------

arxiv_main_category = {
    "astro-ph": [
        "astro-ph.CO", "astro-ph.EP", "astro-ph.GA", "astro-ph.HE",
        "astro-ph.IM", "astro-ph.SR"
    ],
    "cond-mat": [
        "cond-mat.dis-nn", "cond-mat.mes-hall", "cond-mat.mtrl-sci",
        "cond-mat.other", "cond-mat.quant-gas", "cond-mat.soft",
        "cond-mat.stat-mech", "cond-mat.str-el", "cond-mat.supr-con"
    ],
    "gr-qc": [
        "gr-qc"
    ],
    "hep": [
        "hep-ex", "hep-lat", "hep-ph", "hep-th"
    ],
    "math-ph": [
        "math-ph"
    ],
    "nlin": [
        "nlin.AO", "nlin.CG", "nlin.CD", "nlin.SI", "nlin.PS"
    ],
    "nucl": [
        "nucl-ex", "nucl-th"
    ],
    "quant-ph": [
        "quant-ph"
    ],
    "physics": [
        "physics.acc-ph", "physics.ao-ph", "physics.app-ph", "physics.atm-clus",
        "physics.atom-ph", "physics.bio-ph", "physics.chem-ph", "physics.class-ph",
        "physics.comp-ph", "physics.data-an", "physics.ed-ph", "physics.flu-dyn",
        "physics.gen-ph", "physics.geo-ph", "physics.hist-ph", "physics.ins-det",
        "physics.med-ph", "physics.optics", "physics.plasm-ph", "physics.pop-ph",
        "physics.soc-ph", "physics.space-ph"
    ],
    "math": [
        "math.AC", "math.AG", "math.AP", "math.AT", "math.CA", "math.CO",
        "math.CT", "math.CV", "math.DG", "math.DS", "math.FA", "math.GM",
        "math.GN", "math.GR", "math.GT", "math.HO", "math.IT", "math.KT",
        "math.LO", "math.MG", "math.MP", "math.NA", "math.NT", "math.OA",
        "math.OC", "math.PR", "math.QA", "math.RA", "math.RT", "math.SG",
        "math.SP", "math.ST"
    ],
    "cs": [
        "cs.AI", "cs.AR", "cs.CC", "cs.CE", "cs.CG", "cs.CL", "cs.CR", "cs.CV",
        "cs.CY", "cs.DB", "cs.DC", "cs.DL", "cs.DM", "cs.DS", "cs.ET", "cs.FL",
        "cs.GL", "cs.GR", "cs.GT", "cs.HC", "cs.IR", "cs.IT", "cs.LG", "cs.LO",
        "cs.MA", "cs.MM", "cs.MS", "cs.NA", "cs.NE", "cs.NI", "cs.OH", "cs.OS",
        "cs.PF", "cs.PL", "cs.RO", "cs.SC", "cs.SD", "cs.SE", "cs.SI", "cs.SY"
    ],
    "q-bio": [
        "q-bio.BM", "q-bio.CB", "q-bio.GN", "q-bio.MN", "q-bio.NC", "q-bio.OT",
        "q-bio.PE", "q-bio.QM", "q-bio.SC", "q-bio.TO"
    ],
    "q-fin": [
        "q-fin.CP", "q-fin.EC", "q-fin.GN", "q-fin.MF", "q-fin.PM", "q-fin.PR",
        "q-fin.RM", "q-fin.ST", "q-fin.TR"
    ],
    "stat": [
        "stat.AP", "stat.CO", "stat.ME", "stat.ML", "stat.OT", "stat.TH"
    ],
    "eess": [
        "eess.AS", "eess.IV", "eess.SP", "eess.SY"
    ],
    "econ": [
        "econ.EM", "econ.GN", "econ.TH"
    ],
}

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# Create mapping with dictionary comprehension
category_to_parent = {child: parent for parent, children in arxiv_main_category.items() for child in children}

# Define mapping UDF
@udf(StringType())
def map_category(category):
    return category_to_parent.get(category, "other")

# Add column with single function call
papers_with_categories = papers_df.withColumn("arxiv_main_category", map_category(col("main_category")))


In [ ]:
# Get distribution of arxiv_main_category
from pyspark.sql.functions import col, count, round
from pyspark.sql.functions import max as spark_max

# Count the unique arxiv_main_category values
unique_arxiv_main_categories = papers_with_categories.select("arxiv_main_category").distinct().count()
print(f"Number of unique arxiv_main_categories: {unique_arxiv_main_categories}")


# Count categories and calculate percentages
total_papers = papers_with_categories.count()
cat_distribution = papers_with_categories.groupBy("arxiv_main_category") \
    .count() \
    .withColumn("percentage", round((col("count") / total_papers) * 100, 2)) \
    .orderBy("count", ascending=False)

# Display distribution
print("\nArXiv Main Category Distribution:")
cat_distribution.show(truncate=False)

# Check class imbalance
max_count = cat_distribution.agg({"count": "max"}).collect()[0][0]
min_count = cat_distribution.agg({"count": "min"}).collect()[0][0]
print(f"\nClass Imbalance Ratio (max/min): {max_count/min_count:.2f}")

# Count classes with less than 1% representation
rare_classes = cat_distribution.filter("percentage < 1").count()
print(f"Classes with <1% representation: {rare_classes}")

# Optional: Quick visualization using pandas and matplotlib
import pandas as pd
import matplotlib.pyplot as plt

# Convert to pandas for easy plotting
latest_date = papers_df.agg(spark_max("published")).collect()[0][0]
pd_distribution = cat_distribution.toPandas()

# Create horizontal bar chart
plt.figure(figsize=(12, 8))  # Slightly larger figure to accommodate annotations
bars = plt.barh(pd_distribution['arxiv_main_category'], pd_distribution['percentage'])
plt.xlabel('Percentage (%)')
plt.ylabel('Category')
plt.title(f'ArXiv Main Category Distribution (Total: {total_papers:,} papers)\n(Data through {latest_date})', 
         fontsize=14)
# Add count annotations to each bar
for i, bar in enumerate(bars):
    count = pd_distribution.iloc[i]['count']
    plt.text(
        bar.get_width() + 0.3,  # Position slightly to the right of the bar
        bar.get_y() + bar.get_height()/2,  # Vertical center of the bar
        f'{count:,}',  # Format with commas for thousands
        va='center'  # Vertically centered
    )

plt.tight_layout()
plt.show()


# Input features

Evaluate your input features `title` and `summary`:
- What is the average/median/min/max length (in words or characters) of:
    - title
    - summary
    - title + summary?
- Are there unusually short or long texts that should be filtered?
- Are there stopwords or LaTeX/math formatting (\\mathsf, etc.) that should be removed?
- Are there phrases or token patterns in title and summary that can be related to `main_category`?


In [ ]:
from pyspark.sql.functions import length, size, split, col, expr, percentile_approx
from pyspark.sql.functions import min as spark_min, max as spark_max, avg as spark_avg
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.ml.feature import StopWordsRemover, NGram, CountVectorizer, Tokenizer

# ---------- 1. Text Length Statistics ----------
text_stats = papers_df.select(
    length("title").alias("title_chars"),
    size(split("title", " ")).alias("title_words"),
    length("summary").alias("summary_chars"),
    size(split("summary", " ")).alias("summary_words")
).withColumn(
    "combined_chars", col("title_chars") + col("summary_chars")
).withColumn(
    "combined_words", col("title_words") + col("summary_words")
)

# Calculate statistics
stats = text_stats.agg(
    spark_min("title_chars"), spark_max("title_chars"), spark_avg("title_chars"), percentile_approx("title_chars", 0.5),
    spark_min("title_words"), spark_max("title_words"), spark_avg("title_words"),
    spark_min("summary_chars"), spark_max("summary_chars"), spark_avg("summary_chars"), 
    spark_min("summary_words"), spark_max("summary_words"), spark_avg("summary_words"),
    spark_min("combined_chars"), spark_max("combined_chars"), spark_avg("combined_chars"),
    spark_min("combined_words"), spark_max("combined_words"), spark_avg("combined_words")
).collect()[0]

print("\nText Statistics (min, max, avg):")
print(f"Title: {stats[0]}-{stats[1]} chars ({stats[2]:.1f} avg), {stats[4]}-{stats[5]} words ({stats[6]:.1f} avg)")
print(f"Summary: {stats[7]}-{stats[8]} chars ({stats[9]:.1f} avg), {stats[10]}-{stats[11]} words ({stats[12]:.1f} avg)")
print(f"Combined: {stats[13]}-{stats[14]} chars ({stats[15]:.1f} avg), {stats[16]}-{stats[17]} words ({stats[18]:.1f} avg)")


In [ ]:
# ---------- 2. Identify Outliers (1st/99th percentiles) ----------
percentiles = text_stats.select(
    percentile_approx("title_chars", [0.01, 0.99]).alias("title_thresholds"),
    percentile_approx("summary_chars", [0.01, 0.99]).alias("summary_thresholds")
).collect()[0]

print(f"\nOutlier thresholds (1st-99th percentiles):")
print(f"Title length: {percentiles[0][0]}-{percentiles[0][1]} chars")
print(f"Summary length: {percentiles[1][0]}-{percentiles[1][1]} chars")


In [ ]:
# ---------- 3. Check for LaTeX and Math formatting ----------
latex_pattern = r'\\[a-zA-Z]+'
latex_counts = papers_df.select(
    (size(expr(f"regexp_extract_all(title, '{latex_pattern}', 0)")) > 0).cast("int").alias("title_has_latex"),
    (size(expr(f"regexp_extract_all(summary, '{latex_pattern}', 0)")) > 0).cast("int").alias("summary_has_latex")
).agg(
    spark_sum("title_has_latex"), 
    spark_sum("summary_has_latex")
).collect()[0]

print(f"\nLaTeX Usage:")
print(f"Papers with LaTeX in title: {latex_counts[0]} ({latex_counts[0]/papers_df.count()*100:.1f}%)")
print(f"Papers with LaTeX in summary: {latex_counts[1]} ({latex_counts[1]/papers_df.count()*100:.1f}%)")


In [ ]:
from pyspark.sql.functions import explode, col, count as spark_count, trim, lower, regexp_replace

# 1. Better text cleaning before processing
clean_papers = papers_df.withColumn(
    "clean_title", 
    trim(regexp_replace(lower(col("title")), "[^a-zA-Z0-9\\s]", " "))
)

# 2. Apply tokenization pipeline
tokenizer = Tokenizer(inputCol="clean_title", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
bigram = NGram(n=2, inputCol="filtered", outputCol="bigrams")

# 3. Generate and filter meaningful bigrams
pipeline_result = bigram.transform(remover.transform(tokenizer.transform(clean_papers)))
bigram_counts = pipeline_result.select(
    explode("bigrams").alias("bigram")
).filter(
    # Multiple filters to ensure quality bigrams
    (length(trim(col("bigram"))) > 3) &                   # Reasonable length
    (~col("bigram").contains("  ")) &                     # No double spaces
    (col("bigram").rlike("^[a-zA-Z0-9].*[a-zA-Z0-9]$"))   # Start/end with alphanumeric
).groupBy(
    "bigram"
).agg(
    spark_count("*").alias("count")
).orderBy(
    col("count").desc()
)

# 4. Show results
print("\nTop 10 bigrams in titles:")
for row in bigram_counts.limit(10).collect():
    print(f"'{row.bigram}': {row['count']}")
    

In [ ]:
from pyspark.sql.functions import explode, col, count as spark_count

# 1. Process all papers once with category information
category_bigrams = clean_papers.select(
    "main_category", 
    "clean_title"
).transform(
    # Apply the same pipeline to all papers at once
    lambda df: bigram.transform(remover.transform(tokenizer.transform(df)))
)

# 2. Extract, filter and count bigrams with categories in one operation
bigram_by_category = category_bigrams.select(
    "main_category",
    explode("bigrams").alias("bigram")
).filter(
    # Apply same quality filters as main analysis 
    (length(trim(col("bigram"))) > 3) &
    (~col("bigram").contains("  ")) &
    (col("bigram").rlike("^[a-zA-Z0-9].*[a-zA-Z0-9]$"))
).groupBy(
    "main_category", "bigram"
).count().orderBy(
    col("main_category"), col("count").desc()
)

# 3. Display top bigrams for selected categories in one efficient loop
top_categories = papers_df.groupBy("main_category").count() \
                          .orderBy("count", ascending=False) \
                          .limit(10) \
                          .select("main_category") \
                          .rdd.flatMap(lambda x: x).collect()


print("\nTop 5 bigrams per category:")
for category in top_categories[:10]:
    print(f"\n{category}:")
    top_for_category = bigram_by_category.filter(col("main_category") == category).limit(5).collect()
    for row in top_for_category:
        print(f"  '{row.bigram}': {row['count']}")

In [ ]:
from pyspark.sql.functions import explode, col, count as spark_count, trim, lower, regexp_replace, length

# ----------------- Summary Bigram Analysis -----------------

# 1. Clean the summary text
clean_papers_summary = papers_df.withColumn(
    "clean_summary", 
    trim(regexp_replace(lower(col("summary")), "[^a-zA-Z0-9\\s]", " "))
)

# 2. Apply tokenization pipeline
sum_tokenizer = Tokenizer(inputCol="clean_summary", outputCol="summary_tokens")
sum_remover = StopWordsRemover(inputCol="summary_tokens", outputCol="summary_filtered")
sum_bigram = NGram(n=2, inputCol="summary_filtered", outputCol="summary_bigrams")

# 3. Generate and filter meaningful bigrams
summary_pipeline = sum_bigram.transform(
    sum_remover.transform(
        sum_tokenizer.transform(clean_papers_summary)
    )
)

# 4. Extract and count bigrams
summary_bigram_counts = summary_pipeline.select(
    explode("summary_bigrams").alias("bigram")
).filter(
    # Same quality filters as used for titles
    (length(trim(col("bigram"))) > 3) &
    (~col("bigram").contains("  ")) &
    (col("bigram").rlike("^[a-zA-Z0-9].*[a-zA-Z0-9]$"))
).groupBy(
    "bigram"
).agg(
    spark_count("*").alias("count")
).orderBy(
    col("count").desc()
)

# 5. Show top bigrams in summaries
print("\nTop 10 bigrams in summaries:")
for row in summary_bigram_counts.limit(10).collect():
    print(f"'{row.bigram}': {row['count']}")

# 6. Category-specific summary bigrams in a single efficient operation
category_summary_bigrams = clean_papers_summary.select(
    "main_category",
    "clean_summary"
).transform(
    # Apply the pipeline to all papers at once
    lambda df: sum_bigram.transform(sum_remover.transform(sum_tokenizer.transform(df)))
)

# 7. Process all categories at once
summary_bigrams_by_category = category_summary_bigrams.select(
    "main_category",
    explode("summary_bigrams").alias("bigram")
).filter(
    # Same quality filters
    (length(trim(col("bigram"))) > 3) &
    (~col("bigram").contains("  ")) &
    (col("bigram").rlike("^[a-zA-Z0-9].*[a-zA-Z0-9]$"))
).groupBy(
    "main_category", "bigram"
).count().orderBy(
    col("main_category"), col("count").desc()
)

# 8. Display top summary bigrams by category
print("\nTop 5 summary bigrams per category:")
for category in top_categories[:5]:  # Limiting to top 5 categories for brevity
    print(f"\n{category}:")
    top_for_category = summary_bigrams_by_category.filter(col("main_category") == category).limit(5).collect()
    for row in top_for_category:
        print(f"  '{row.bigram}': {row['count']}")
        

In [ ]:
# Are certain keywords more correlated with certain categories?

from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col, expr, udf
from pyspark.sql.types import FloatType, ArrayType, StringType
from pyspark.ml import Pipeline

# 1. Prepare data - clean text and add category
keyword_data = papers_df.select(
    "main_category",
    regexp_replace(lower(col("title")), "[^a-zA-Z0-9\\s]", " ").alias("cleaned_text")
)

# 2. Create ML pipeline for feature extraction
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])

# 3. Fit the pipeline and transform data
model = pipeline.fit(keyword_data)
transformed = model.transform(keyword_data)

# 4. Calculate keyword significance for each category
# Get vocabulary from the HashingTF model (using the vocabulary from token counts)
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="filtered", outputCol="counts", minDF=10)
cv_model = cv.fit(transformed)
vocab = cv_model.vocabulary

# 5. For each category, find most distinctive keywords
print("\nTop keywords correlated with categories:")
for category in top_categories[:10]:  # Show top 20 categories
    # Filter for this category and collect weighted term frequencies
    category_docs = transformed.filter(col("main_category") == category)
    
    # Extract top terms for this category using the TF-IDF weights
    @udf(returnType=ArrayType(StringType()))
    def get_top_terms(words, features):
        # Pair words with their feature weights and sort by weight
        return [w for _, w in sorted([(features[i], w) for i, w in enumerate(words) if i < len(features)], 
                                   key=lambda x: x[0], reverse=True)[:10]]
    
    # Apply the UDF to get top terms for each document
    category_terms = category_docs.withColumn("top_terms", get_top_terms("filtered", "features"))
    
    # Count term frequency across all documents in this category
    top_terms = category_terms.select(explode("filtered").alias("term")).groupBy("term").count()
    
    # Get overall corpus frequency for comparison
    corpus_freq = transformed.select(explode("filtered").alias("term")).groupBy("term").count().withColumnRenamed("count", "corpus_count")
    
    # Calculate term importance as ratio of category frequency to corpus frequency
    term_importance = top_terms.join(corpus_freq, "term").withColumn(
        "importance", col("count") / col("corpus_count")
    ).orderBy("importance", ascending=False)
    
    # Display results
    print(f"\n{category}:")
    for row in term_importance.filter(col("count") > 5).limit(10).collect():
        print(f"  '{row.term}': {row['count']} occurrences, {row.importance:.2f} importance")
